# Sensitivity Analysis of NPIs used a Hybrid LSTM-SIR model

### Here, an analysis will be done to identify the sensitivity of OxCGRT-variables (non-pharmaceutical interventions, NPIs). In particular, sensitivity coefficient (E) will be computed to determine the impact of changes in the dataset and how sensitive model predictions are to missing data or a small input dataset.

In [1]:
import os
os.chdir("../LSTM")

In [2]:
import covsirphy as cs

import pandas as pd
import plotly.graph_objects as go
from core.sensitivityanalysis import SA
%load_ext autoreload
%autoreload 2

In [3]:
# Model parameters
parameters = ["window", "steps", "delay", "clustering threshold"]

# NPI-variables
interventions = ['Stringency_index', 'School_closing', 'Workplace_closing', 'Cancel_events',
 'Gatherings_restrictions', 'Transport_closing', 'Stay_home_restrictions',
 'Internal_movement_restrictions', 'International_movement_restrictions',
 'Information_campaigns', 'Testing_policy', 'Contact_tracing']

# Select country
COUNTRY = "Italy"

# Download datasets
data_loader = cs.DataLoader("input")
jhu_data = data_loader.jhu()
population_data = data_loader.population()
oxcgrt_data = data_loader.oxcgrt()
s = cs.Scenario(jhu_data, population_data, country=COUNTRY)
days_delay, df_periods = s.estimate_delay(oxcgrt_data)
oxcgrt_data = oxcgrt_data.cleaned()
jhu_data = jhu_data.cleaned()
population_data = population_data.cleaned()

Retrieving datasets from COVID-19 Data Hub: https://covid19datahub.io/

Please set verbose=2 to see the detailed citation list.


Retrieving COVID-19 dataset in Japan from https://github.com/lisphilar/covid19-sir/data/japan


In [ ]:
input_rates = [0.1, 0.5]
PARAMS = {
    "window": 7,
    "steps": 5,
    "days_delay": days_delay,
    "df_periods": df_periods,
    "TARGET": interventions[0],
    "input_rate": input_rates[0]
}

results_intervention = {}
for intervention in interventions:
    results = {}
    for input_rate in input_rates:
        PARAMS["TARGET"] = intervention
        PARAMS["input_rate"] = input_rate
        print(input_rate, intervention)
        try:
            errors = SA(COUNTRY, PARAMS, oxcgrt_data, jhu_data, population_data)
        except: 
            print("Not enough data for the NPI ", intervention)
            continue 
        results[input_rate] = errors
    results_intervention[intervention] = results


In [ ]:
res = results_intervantion.copy()
concat_dfs = []
strings1 = list(res.keys())
new_strings1 = [string + "0.1" for string in strings1]
df_1 = pd.DataFrame(columns = new_strings1)
for npi in res:
    df_1[npi + '0.1'] = res[npi][0.1]

strings5 = list(res.keys())
new_strings5 = [string + "0.5" for string in strings1]
df_5 = pd.DataFrame(columns = new_strings5)
for npi in res:
    df_5[npi + '0.5'] = res[npi][0.5]

total = pd.concat([df_1, df_5], axis=1)
concat_dfs.append(total)

In [ ]:
means = pd.concat(concat_dfs)

diff = pd.DataFrame()
for col in interventions:
    name = col.replace("_", " ")
    if name == "Stringency index":
        name = "Stringency index / Lockdown"

    diff[name] = abs(means[col + '0.1'] - means[col + '0.5'])

diff = diff.reindex(diff.mean().sort_values().index, axis=1)

fig = go.Figure()
for col in diff:
    fig.add_trace(go.Box(x=diff[col].values, name=diff[col].name, marker_color="orange"))
    
fig.update_xaxes(title_text="<b>Sensitivity Coefficiënt E</b>")
fig.update_layout(template="none", title="NPI Sensitivity analysis of M1")

fig.update_layout(
    autosize=True,
    width=900,
    height=500,
    margin=go.layout.Margin(
        l=200
    )
    )
fig.layout.font.family = 'Arial'
fig.update_layout(showlegend=False)
fig.show()


![Sensitivity Analysis](../SIR/SA_M1.png)
